In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
print(len(train_data))
display(train_data.isnull().sum())
display(test_data)
print(train_data.groupby(['Pclass','Survived'])['PassengerId'].count())
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
891


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Pclass  Survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
Name: PassengerId, dtype: int64


First, I will explore the basic facts of the training data, identifying the names, datatypes, and number of missing values in each of the colums.

In [2]:
print(len(train_data))
print(train_data.columns)
print(train_data.dtypes)
display(train_data.isnull().sum())
print(type(train_data))

891
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

<class 'pandas.core.frame.DataFrame'>


Most of the cabin data is missing. I will remove this column.

In [3]:
train_data.drop(columns = ['Cabin'], inplace = True, axis = 1)


I will build a baseline model just by inspecting the data. This will be helpful to have something to compare more advanced machine learning methods against. My first guess is that the ticket class will be helpful in predicting the survival of the guests.

In [4]:
print(train_data.groupby(['Pclass','Survived'])['PassengerId'].count())


Pclass  Survived
1       0            80
        1           136
2       0            97
        1            87
3       0           372
        1           119
Name: PassengerId, dtype: int64


We see that first class passengers have a relatively high likelihood of survival, whereas third class passengers have a much lower probability of surving. Second class passengers are pretty close to 50/50. I'll try to find another predictive variable to split the second class passengers.

In [5]:
train_data_second_class = train_data[train_data['Pclass'] == 2]
train_data_second_class.groupby('Sex')['Survived'].value_counts()


Sex     Survived
female  1           70
        0            6
male    0           91
        1           17
Name: count, dtype: int64

We see that women in the second class survived at a much higher rate. This leads to our

In [6]:
preds = []
for i in range(len(test_data)):
    if test_data['Pclass'].iloc[i] == 1:
        preds.append(1)
    elif test_data['Pclass'].iloc[i] == 3:
        preds.append(0)
    else:
        if test_data['Sex'].iloc[i] == 'female':
            preds.append(1)
        else:
            preds.append(0)
test_data['Survived'] = preds
submission = test_data[['PassengerId','Survived']]
submission.to_csv('submission.csv', index = False)

Predictably, this model didn't do particularly well. It ended up scoring ~72%. We will now try to use the data to build a more sophisticated machine learning model. First, we will translate the categorical variables to numerical variables.

In [7]:
train_data = pd.get_dummies(train_data, columns = ['Pclass','Sex', 'Embarked'], drop_first = True)

print(train_data['Age'].median())
train_data['Age'].fillna(value = train_data['Age'].median(), inplace = True )
display(train_data.isna().sum())
print(len(train_data))

28.0


PassengerId    0
Survived       0
Name           0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Pclass_2       0
Pclass_3       0
Sex_male       0
Embarked_Q     0
Embarked_S     0
dtype: int64

891


In [8]:

X = train_data.drop(columns = ['Survived','Name','PassengerId','Ticket'], axis = 1)
y = train_data['Survived']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 27)
display(X_train)
display(y_test)

,Age,SibSp,Parch,Fare,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
287,22.0,0,0,7.8958,False,True,True,False,True
220,16.0,0,0,8.0500,False,True,True,False,True
643,28.0,0,0,56.4958,False,True,True,False,True
542,11.0,4,2,31.2750,False,True,False,False,True
120,21.0,2,0,73.5000,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...
824,2.0,4,1,39.6875,False,True,True,False,True
543,32.0,1,0,26.0000,True,False,True,False,True
840,20.0,0,0,7.9250,False,True,True,False,True
696,44.0,0,0,8.0500,False,True,True,False,True


432    1
10     1
38     0
747    1
314    0
      ..
746    0
587    1
66     1
99     0
321    0
Name: Survived, Length: 223, dtype: int64

We will start by training a k nearest neighbors model.

In [9]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
test_data_clean = test_data.drop(columns = ['PassengerId','Name','Ticket','Cabin','Survived'], axis = 1)
test_data_clean = pd.get_dummies(test_data_clean, columns = ['Pclass','Sex', 'Embarked'], drop_first = True)
test_data_clean['Age'].fillna(value = 28.0, inplace = True)
test_data_clean['Fare'].fillna(value = train_data['Fare'].median(), inplace = True)
ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

params = {'n_neighbors':[1,2,3,4,5,6]}

knn = KNN()
gscv = GridSearchCV(estimator = knn, param_grid = params, scoring = 'accuracy', cv = 5)
gscv.fit(X_train_scaled,y_train)
predictions = gscv.predict(X_test_scaled)
print(accuracy_score(predictions,y_test))
print(gscv.best_params_)
preds = gscv.predict(ss.transform(test_data_clean))
test_data['Survived'] = preds
test_data['Survived_knn'] = preds
knn_submission = test_data[['PassengerId','Survived']]
knn_submission.to_csv('knnsubmission.csv', index = False)

0.8071748878923767
{'n_neighbors': 6}


The model performs reasonably well (about 77% on the actual test data), but there is definitely room for improvement. We will try a decision tree classifier next. First we will just try a specific set of parameters before trying to optimize with a full grid search.

In [10]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth = 5, min_samples_leaf = .1)
dtc.fit(X_train,y_train)
predictions = dtc.predict(X_test)
print(accuracy_score(predictions,y_test))

0.757847533632287


This is unlikely to outperform the KNN model, so we will try to build a more accurate model with gridsearchCV.

In [11]:
params = {'max_depth' :range(2,9), 'min_samples_leaf' : np.linspace(0.01,.99,20)}
dt = DecisionTreeClassifier()
gscv = GridSearchCV(estimator = dt, param_grid = params, scoring ='accuracy', cv = 5)
gscv.fit(X_train,y_train)
predictions = gscv.predict(X_test)
print(accuracy_score(predictions,y_test))

preds = gscv.predict(test_data_clean)
test_data['Survived'] = preds
test_data['Survived_dt'] = preds
dtc_submission = test_data[['PassengerId','Survived']]
dtc_submission.to_csv('dtc_submssion.csv', index = False)

0.8071748878923767


On the hidden test data, this model scored about 76% so slightly worse than the knn approach. Next we will try an extreme gradient boosting approach.

In [12]:
import xgboost

xgb = xgboost.XGBClassifier(objective = 'binary:logistic')
param_grid = {'n_estimators':range(3,15)}
xgbgscv = GridSearchCV(estimator = xgb, param_grid = param_grid, scoring = 'accuracy', cv = 5)
xgbgscv.fit(X_train,y_train)
predictions = xgbgscv.predict(X_test)
print(accuracy_score(y_test,predictions))
preds = xgbgscv.predict(test_data_clean)
test_data['Survived'] = preds
test_data['Survived_xgb'] = preds
xgb_submission = test_data[['PassengerId','Survived']]
xgb_submission.to_csv('xgb_submission.csv',index = False)
print(xgbgscv.best_params_)

0.820627802690583
{'n_estimators': 7}


This model also scored around a 77% on the hidden data. Next we will try a random forest approach.

In [13]:
#from sklearn.ensemble import RandomForestClassifier
#rf = RandomForestClassifier()
#params = {'n_estimators': [50,100,150], 'max_depth':[4,5,6,7,8], 'min_samples_leaf':[.05,.1,.15,.2]}
#rfgscv = GridSearchCV(rf, param_grid = params, scoring = 'accuracy', cv = 5)
#rfgscv.fit(X_train,y_train)
#predictions = rfgscv.predict(X_test)
#print(accuracy_score(y_test,predictions))
#preds = rfgscv.predict(test_data_clean)
#test_data['Survived'] = preds
#test_data['Survived_rf'] = preds
#rf_submission = test_data[['PassengerId','Survived']]
#rf_submission.to_csv('rf_submission.csv', index = False)
#print(rfgscv.best_params_)

This model found that the best score for the minimum samples was 0.05, so I will try again with even lower values for min sample leaf to see if the perforance can improve further.

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
params = {'n_estimators': [75,100,125], 'max_depth':[5], 'min_samples_leaf':[.05,.01,.015]}
rfgscv = GridSearchCV(rf, param_grid = params, scoring = 'accuracy', cv = 5)
rfgscv.fit(X_train,y_train)
predictions = rfgscv.predict(X_test)
print(accuracy_score(y_test,predictions))
preds = rfgscv.predict(test_data_clean)
test_data['Survived'] = preds
test_data['Survived_rf'] = preds
rf_submission = test_data[['PassengerId','Survived']]
rf_submission.to_csv('rf_submission.csv', index = False)
print(rfgscv.best_params_)

0.8295964125560538
{'max_depth': 5, 'min_samples_leaf': 0.01, 'n_estimators': 100}


In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_scaled,y_train)
preds = lr.predict(X_test_scaled)
print(accuracy_score(y_test,preds))
predictions = lr.predict(ss.transform(test_data_clean))
test_data['Survived'] = predictions
test_data['Survived_lr'] = predictions
lr_submission = test_data[['PassengerId','Survived']]
lr_submission.to_csv('lr_submission.csv', index = False)

0.7982062780269058


All of the submissions seem to hover around 77% accuracy. My next attempt will try to combine all five of the previous models by taking a majority decision from all five predictions.

In [16]:
ensemble_preds = []
for i in range(len(test_data)):
    if test_data['Survived_knn'].iloc[i] + test_data['Survived_xgb'].iloc[i] + test_data['Survived_rf'].iloc[i] + test_data['Survived_dt'].iloc[i] + test_data['Survived_lr'].iloc[i] >2:
        ensemble_preds.append(1)
    else:
        ensemble_preds.append(0)
test_data['Survived'] = ensemble_preds
ensemble_submission = test_data[['PassengerId','Survived']]
ensemble_submission.to_csv('bagged_submission.csv', index = False)

This model hits an accuracy of about 78%.